In [30]:
from pprint import pprint

keywords = ['anyOf', 'properties', 'not', 'const', 'fields', 'anyOf', 'allOf', 'oneOf', 'if', 'then', 'else', 'enum'
           '$id', '$schema', 'title', 'type', 'minimum', 'maximum', 'pattern', 'required', 'items', 'enum', 'prefixItems']
 
def add_prop(json_value):
    '''add "properties" keyword for JSON Schema check'''
    if isinstance(json_value, list):
        return [add_prop(val) for val in json_value]
    if isinstance(json_value, dict):
        prop = {k[1:]: add_prop(v) for k,v in json_value.items() if k[0] == '/'}
        return {k: add_prop(v) for k,v in json_value.items() if k[0] != '/'
               } | ({'properties': prop} if prop else prop)
    return json_value.split('/', maxsplit=1)[-1] if isinstance(json_value, str) else json_value

def del_prop(json_value):
    if isinstance(json_value, list):
        return [del_prop(val) for val in json_value]
    if isinstance(json_value, dict):
        if 'required' in json_value:
            json_value['required'] = ['/' + val for val in json_value['required']]
        if 'properties' in json_value:
            json_value |= {'/' + k: v for k, v in json_value['properties'].items()}
            del(json_value['properties'])
        return {k: del_prop(v) for k, v in json_value.items()}
    return json_value

In [2]:

complex = {
  "$id": "https://example.com/complex-object.schema.json",
  "$schema": "https://json-schema.org/draft/2020-12/schema",
  "title": "Complex Object",
  "type": "object",
  "properties": {
    "name": {
        "type": "string"},
    "age": {
        "type": "integer",
        "minimum": 0},
    "address": {
        "type": "object",
        "properties": {
            "street": {
                "type": "string"},
            "city": {
                "type": "string"},
            "state": {
                "type": "string"},
            "postalCode": {
                "type": "string",
                "pattern": "\\d{5}"}},
        "required": ["street", "city", "state", "postalCode"]},
    "hobbies": {
        "type": "array",
        "items": {
            "type": "string"}}},
  "required": ["name", "age"]}

In [31]:
pprint(del_prop(complex_p), sort_dicts=False)

{'$id': 'https://example.com/complex-object.schema.json',
 '$schema': 'https://json-schema.org/draft/2020-12/schema',
 'title': 'Complex Object',
 'type': 'object',
 '/name': {'type': 'string'},
 '/age': {'type': 'integer', 'minimum': 0},
 '/address': {'type': 'object',
              '/street': {'type': 'string'},
              '/city': {'type': 'string'},
              '/state': {'type': 'string'},
              '/postalCode': {'type': 'string', 'pattern': '\\d{5}'},
              'required': ['//street', '//city', '//state', '//postalCode']},
 '/hobbies': {'type': 'array', 'items': {'type': 'string'}},
 'required': ['///name', '///age']}


In [4]:
complex_p = {
    "$id": "https://example.com/complex-object.schema.json",
    "$schema": "https://json-schema.org/draft/2020-12/schema",
    "title": "Complex Object",
    "type": "object",
    "/name": {
        "type": "string"},
    "/age": {
        "type": "integer",
        "minimum": 0},
    "/address": {
        "type": "object",
        "/street": {
            "type": "string"},
        "/city": {
            "type": "string"},
        "/state": {
            "type": "string"},
        "/postalCode": {
            "type": "string",
            "pattern": "\\d{5}"},
        "required": ["/street", "/city", "/state", "/postalCode"]},
    "/hobbies": {
        "type": "array",
        "items": {
            "type": "string"}},
    "required": ["/name", "/age"]}

In [26]:
pprint(add_prop(complex_p), sort_dicts=False)

{'$id': '/example.com/complex-object.schema.json',
 '$schema': '/json-schema.org/draft/2020-12/schema',
 'title': 'Complex Object',
 'type': 'object',
 'required': ['name', 'age'],
 'properties': {'name': {'type': 'string'},
                'age': {'type': 'integer', 'minimum': 0},
                'address': {'type': 'object',
                            'required': ['street',
                                         'city',
                                         'state',
                                         'postalCode'],
                            'properties': {'street': {'type': 'string'},
                                           'city': {'type': 'string'},
                                           'state': {'type': 'string'},
                                           'postalCode': {'type': 'string',
                                                          'pattern': '\\d{5}'}}},
                'hobbies': {'type': 'array', 'items': {'type': 'string'}}}}


In [3]:
array = {
  "type": "array",
  "prefixItems": [
    { "type": "number" },
    { "type": "string" },
    { "enum": ["Street", "Avenue", "Boulevard"] },
    { "enum": ["NW", "NE", "SW", "SE"] }]}

array_p = {
  "type": "array",
  "/0": 
    { "type": "number" },
  "/1":    
    { "type": "string" },
  "/2": 
    { "enum": ["Street", "Avenue", "Boulevard"] },
  "/3": 
    { "enum": ["NW", "NE", "SW", "SE"] }}